In [68]:
import pandas as pd
import re
import numpy
from bs4 import BeautifulSoup
import os
import requests
pd.options.mode.chained_assignment = None  


def step1(text):
    reg = '<DOCUMENT>\n<TYPE>10-K[\s\S]*?</DOCUMENT>'
    match = re.search(reg, text, flags=re.IGNORECASE)
    if not match:
        return None
    return text[match.start():match.end()]
    
def step2(text):
    reg = '<TEXT>[\s\S]*?</TEXT>'
    match = re.search(reg, text, flags=re.IGNORECASE)
    if not match:
        return None
    text = text[match.start():match.end()]
    return text[6:-7]

def get_html(text):
    r1 = step1(text)
    if not r1:
        return None
    r2 = step2(r1)
    if not r2:
        return None
    return r2

def replace_ascii_code(text, code_map):
    for k, v in code_map.items():
        text = text.replace(k, v)
    return text
    
def remove_href(text):
    soup = BeautifulSoup(text)
    for a in soup.findAll('a'):
        a.decompose()
    return soup.text

def fix_format(text):
    regs = {
        'Risk(\n|\s)*Factors': 'Risk Factors ',
        'Item(\n|\s)*1A': 'Item 1A ',
        'Item(\n|\s)*1B': 'Item 1B '}
    for k, v in regs.items():
        text = re.sub(k, v, text, re.IGNORECASE)
    
    return text

def get_risk_factors(text):
    reg = 'Item(\n|\s)*1A[^a-z]*Risk Factors[\s\S]*?Item 1B'
    match = re.search(reg, text, flags=re.IGNORECASE)
    if not match:
        return None
    return text[match.start():match.end()]

In [69]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

url = 'https://www.tutorialspoint.com/html/html_ascii_codes.htm'
r = requests.get(url, headers=header)
dfs = pd.read_html(r.text)
df = dfs[0].iloc[1:]
df.loc[df[2] == '&#32;', 0] = ''
df.loc[df[2] == '&#44;', 0] = ':'
code_map = df[[2, 0]].set_index(2).to_dict()[0]


url = 'https://www.tutorialspoint.com/html/html_color_names.htm'
r = requests.get(url, headers=header)
dfs = pd.read_html(r.text)
df = dfs[5]
df[0] = df[0].fillna(' ')
df = df.iloc[1:]
code_map.update(df[[3, 0]].set_index(3).to_dict()[0])

code_map.update({
    #'\n': ' ', 
    #'\r': ' ', 
    '&nbsp;': ' ', 
    '&#160;': ' ', 
    '&reg;': '',
    '&#149;': '',
    '&#151;': '—',
    '&amp;': '',
    '&#253;': '',
    '&#153;': '',
    '&#147;': '"',
    '&#148;': '"',
    '&#252;': '',
    '&#167;': '',
    '&#146;': "'",
    '&ccedil;': '',
    '&atilde;': '',
    '&#150;': '-',
    '&agrave;': '',
    '&eacute;': '',
    '&#361;': ''})

del code_map['&#92;']

In [70]:
source_path = './data/tenk/'
target_path = './data/itemone/'

In [71]:
files = os.listdir(source_path)

In [72]:
for file in files:
    t = get_html(open(source_path + file).read())
    if not t:
        print(source_path + file, '          step 1')
    
    t = replace_ascii_code(t, code_map)
    t = remove_href(t)
    t = fix_format(t)
    t = get_risk_factors(t)
    if not t:
        print(source_path + file, '          step 2')
        break
    open(target_path + file, 'w').write(t)

/home/wzh/miniconda3/envs/edgar/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/wzh/miniconda3/envs/edgar/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


./data/tenk/65984-0000065984-17-000067.txt           step 2


In [81]:
t = get_html(open('./data/tenk/65984-0000065984-17-000067.txt').read())
if not t:
    print(source_path + file, '          step 1')

t = replace_ascii_code(t, code_map)

t = re.sub(r'<SMALL>', '', t, flags=re.IGNORECASE)
t = re.sub(r'</SMALL>', '', t, flags=re.IGNORECASE)

# t = remove_href(t)
# t = fix_format(t)

In [82]:
soup = BeautifulSoup(t, 'lxml')

In [84]:
for a in soup.findAll('a'):
    a.decompose()

In [86]:
open('temp.txt', 'w').write(soup.text)

1748337